In [1]:
import json
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from pprint import pprint

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

from datasets import load_dataset, load_from_disk
from transformers import (
    AutoTokenizer,
    BertModel, BertPreTrainedModel,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments, AutoModel,
)

In [2]:
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.7.1].
device:[cuda:0].


In [17]:
train_dataset = load_from_disk("../data/train_dataset/train")
sample_idx = np.random.choice(range(len(train_dataset)), 1000)
training_dataset = train_dataset[sample_idx]
print(training_dataset['context'][0])
print(training_dataset['question'][0])
sample_idx

우왕은 어린 시절을 신돈의 집에서 보내야 했다. 공민왕은 원래 자식이 없어 고민하였는데 신돈이 자신의 여종인 반야를 바쳐 아이를 얻으라고 권유하였다. 이에 공민왕은 반야와 동침했고 얼마 뒤에 반야는 임신하였다. 반야가 만삭이 되자 신돈은 자신의 친구인 승려 능우(能禑)의 어머니에게 반야를 맡겼다 능우의 어머니 집에서 아이를 출산한 반야는 일년 후에 신돈의 집에 가서 기거하였다. 신돈은 동지밀직 김횡이 보낸 여종 김장을 유모로 삼아 아이를 돌보게 했다\n\n반야는 신돈의 여종이었고 공민왕은 반야의 아이를 신돈의 아이라고 할까 봐 근심하고 고민하였다. 1371년 신돈이 역모죄로 몰려 수원부(水原府)로 유배되자 공민왕은 자신에게 아들이 있다고 백관들에게 밝히고 반야의 아들 모니노(牟尼奴)를 궁궐로 데려오라고 하였다 공민왕은 자신이 살해당하던 달인 1374년 9월 초, 모니노가 반야의 소생이라고 하면 사람들이 모니노를 신돈의 자식이라고 의심할 것이라 염려한 공민왕은 이미 사망하고 없는 궁인 한씨를 우왕의 생모라고 말하고서 한씨의 삼대(三代) 조상과 그 여자의 외조에게 벼슬을 추증한다 우왕 즉위 후 한씨에게는 순정왕후라는 시호가 내려진다
우왕은 어디서 태어났는가?


array([ 616, 3778, 3640, 3662, 1172,  849, 1507, 2192,  137, 2480,  454,
        940,  173,  925, 3741, 2540, 3077, 3889,  433, 1558,  137, 1393,
       3658, 1847, 3777, 2696,   29, 1150, 3787, 2092, 2789, 1603,  909,
       3280,  369, 3549, 2848,  385, 3918, 2341,  107, 3837, 3247, 1092,
        413, 2479, 3615,  376,  407,    6,  804, 3402, 2479, 2590,  607,
       1864, 1953, 2747, 2676,    1, 1378, 1011,  763, 2479,  902,  148,
       3312,  968,  600, 3832, 3748, 1192, 1851, 1521, 2896, 3276, 1122,
       2803, 2826, 3742,  389, 3145, 3478,  375, 1840, 2358, 2633,  869,
       1188, 3451, 1570, 2852,  460,  547, 1317, 2623, 1704, 2625,  697,
       3498, 2739,  440, 3083, 1777, 1499, 3794,  904, 3325, 1676, 1078,
       3453, 1771,  307, 2205, 3468,   51, 1419, 2976,  777, 3572, 1015,
         66, 2034, 2852, 1411, 2094, 1658, 2901, 2802,  142,  298,  545,
        883, 3665, 2294,  348, 3589, 2927,  781, 3076,  888,  611, 1244,
       1415, 2424,  257, 2533, 3885, 1020, 1509, 18

In [18]:
class RobertaEncoder(AutoModel):
    def __init__(self, config):
        super(RobertaEncoder, self).__init__(config)

        self.roberta = AutoModel(config)
        self.init_weights()
      
    def forward(self, input_ids,  attention_mask=None): 

        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        
        pooled_output = outputs[1]
        return pooled_output

In [19]:
model_checkpoint = "xlm-roberta-base"
p_encoder = RobertaEncoder.from_pretrained(model_checkpoint).to(device)
q_encoder = RobertaEncoder.from_pretrained(model_checkpoint).to(device)

# if torch.cuda.is_available():
#     p_encoder.cuda()
#     q_encoder.cuda()

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [21]:
class DenseRetrieval:
    def __init__(self, args, dataset, num_neg, tokenizer, p_encoder, q_encoder):

        self.args = args
        self.dataset = dataset
        self.num_neg = num_neg

        self.tokenizer = tokenizer
        self.p_encoder = p_encoder
        self.q_encoder = q_encoder

        self.prepare_in_batch_negative(num_neg=num_neg)
        
        # self.eval_dataset = eval_dataset
        # self.prepare_eval_dataset

    def prepare_in_batch_negative(self, dataset=None, num_neg=2, tokenizer=None):

        if dataset is None:
            dataset = self.dataset

        if tokenizer is None:
            tokenizer = self.tokenizer

        # 1. In-Batch-Negative 만들기
        # CORPUS를 np.array로 변환해줍니다.
        corpus = np.array(list(set([example for example in dataset["context"]])))
        p_with_neg = []

        for c in dataset["context"]:
            while True:
                neg_idxs = np.random.randint(len(corpus), size=num_neg)

                if not c in corpus[neg_idxs]:
                    p_neg = corpus[neg_idxs]

                    p_with_neg.append(c)
                    p_with_neg.extend(p_neg)
                    break

        # 2. (Question, Passage) 데이터셋 만들어주기
        q_seqs = tokenizer(dataset["question"], padding="max_length", truncation=True, return_tensors="pt")
        p_seqs = tokenizer(p_with_neg, padding="max_length", truncation=True, return_tensors="pt")

        max_len = p_seqs["input_ids"].size(-1)
        p_seqs["input_ids"] = p_seqs["input_ids"].view(-1, num_neg+1, max_len)
        p_seqs["attention_mask"] = p_seqs["attention_mask"].view(-1, num_neg+1, max_len)
        # p_seqs["token_type_ids"] = p_seqs["token_type_ids"].view(-1, num_neg+1, max_len)

        train_dataset = TensorDataset(
            p_seqs["input_ids"], p_seqs["attention_mask"], # p_seqs["token_type_ids"], 
            q_seqs["input_ids"], q_seqs["attention_mask"], # q_seqs["token_type_ids"]
        )

        self.train_dataloader = DataLoader(
            train_dataset,
            shuffle=True,
            batch_size=self.args.per_device_train_batch_size
        )

        valid_seqs = tokenizer(
            dataset["context"],
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        passage_dataset = TensorDataset(
            valid_seqs["input_ids"],
            valid_seqs["attention_mask"],
            # valid_seqs["token_type_ids"]
        )
        self.passage_dataloader = DataLoader(
            passage_dataset,
            batch_size=self.args.per_device_train_batch_size
        )


    def train(self, args=None):
        if args is None:
            args = self.args
        batch_size = args.per_device_train_batch_size
        print(args.device)

        # Optimizer
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {"params": [p for n, p in self.p_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.p_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
            {"params": [p for n, p in self.q_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.q_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
        ]
        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=args.learning_rate,
            eps=args.adam_epsilon
        )
        t_total = len(self.train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=args.warmup_steps,
            num_training_steps=t_total
        )

        # Start training!
        global_step = 0

        self.p_encoder.zero_grad()
        self.q_encoder.zero_grad()
        torch.cuda.empty_cache()

        train_iterator = tqdm(range(int(args.num_train_epochs)), desc="Epoch")
        # for _ in range(int(args.num_train_epochs)):
        for _ in train_iterator:

            with tqdm(self.train_dataloader, unit="batch") as tepoch:
                for batch in tepoch:

                    p_encoder.train()
                    q_encoder.train()
            
                    targets = torch.zeros(batch_size).long() # positive example은 전부 첫 번째에 위치하므로
                    targets = targets.to(args.device)

                    p_inputs = {
                        "input_ids": batch[0].view(batch_size * (self.num_neg + 1), -1).to(args.device),
                        "attention_mask": batch[1].view(batch_size * (self.num_neg + 1), -1).to(args.device),
                        # "token_type_ids": batch[2].view(batch_size * (self.num_neg + 1), -1).to(args.device)
                    }
            
                    q_inputs = {
                        "input_ids": batch[2].to(args.device),
                        "attention_mask": batch[3].to(args.device),
                        # "token_type_ids": batch[5].to(args.device)
                    }

                    # (batch_size*(num_neg+1), emb_dim)
                    p_outputs = self.p_encoder(**p_inputs)[1]
                    # (batch_size, emb_dim)
                    q_outputs = self.q_encoder(**q_inputs)[1]

                    # Calculate similarity score & loss
                    p_outputs = p_outputs.view(batch_size, -1, self.num_neg+1)
                    q_outputs = q_outputs.view(batch_size, 1, -1)

                    sim_scores = torch.bmm(q_outputs, p_outputs).squeeze()  #(batch_size, num_neg + 1)
                    sim_scores = sim_scores.view(batch_size, -1)
                    sim_scores = F.log_softmax(sim_scores, dim=1)

                    loss = F.nll_loss(sim_scores, targets)
                    tepoch.set_postfix(loss=f"{str(loss.item())}")

                    loss.backward()
                    optimizer.step()
                    scheduler.step()

                    self.p_encoder.zero_grad()
                    self.q_encoder.zero_grad()

                    global_step += 1

                    torch.cuda.empty_cache()

                    del p_inputs, q_inputs


    def get_relevant_doc(self, query, k=1, args=None, p_encoder=None, q_encoder=None):
    
        if args is None:
            args = self.args

        if p_encoder is None:
            p_encoder = self.p_encoder

        if q_encoder is None:
            q_encoder = self.q_encoder

        with torch.no_grad():
            p_encoder.eval()
            q_encoder.eval()

            q_seqs_val = self.tokenizer(
                [query],
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            ).to(args.device)
            q_emb = q_encoder(**q_seqs_val)[1].to("cpu")  # (num_query=1, emb_dim)

            p_embs = []
            for batch in tqdm(self.passage_dataloader):

                batch = tuple(t.to(args.device) for t in batch)
                p_inputs = {
                    "input_ids": batch[0],
                    "attention_mask": batch[1],
                    # "token_type_ids": batch[2]
                }
                p_emb = p_encoder(**p_inputs)[1].to("cpu")
                p_embs.append(p_emb)

        # (num_passage, emb_dim)
        p_embs = torch.stack(p_embs, dim=0).view(len(self.passage_dataloader.dataset), -1)

        dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
        rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
        print(dot_prod_scores)
        print(rank)

        return rank[:k]


In [24]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [25]:
retrieval = DenseRetrieval(args=args, 
                           dataset=training_dataset,
                           # eval_dataset=None, 
                           num_neg=2, 
                           tokenizer=tokenizer, 
                           p_encoder=p_encoder, 
                           q_encoder=q_encoder
                          )

In [26]:
retrieval.train()

cuda:0


In [27]:
IDX = 10
print(training_dataset['question'][IDX])
print(training_dataset['context'][IDX])

링컨이 조지 워싱턴 장군에 합세하여 우익을 통솔했던 전투는?
1776년, 링컨은 준장에 이어 소장으로 승격되어, 보스턴 지역의 모든 매사추세츠 부대를 지휘했다. 영국군이 보스턴에서 철수한 후 링컨은 뉴욕에서 조지 워싱턴 장군에 합류, 화이트 플래인스 전투에서는 우익을 지휘했다. 인디펜던스 요새 전투 직후에 링컨은 대륙군 소장이 되었다.\n\n1777년 9월, 링컨은 새러토가 근처에 있던 호레이쇼 게이츠의 부대에 합류하여 새러토가 전투에 참전했다. 그러나 뒤꿈치에 머스켓 총알을 맞은 상처 때문에 새러토가 전투에서는 큰 역할을 하지 못했다. 이때의 상처로 한쪽 다리가 짧은 상태가 되었다. 이 상처가 아물자 링컨은 1778년 9월에 남부 방면 군의 지휘관에 임명되었다. 링컨은 1779년 10월 9일의 조지아의 사바나 공격에 참가, 사우스캐롤라이나의 찰스턴으로 철수를 하도록 만들었다. 링컨에게는 운이 나빴지만, 찰스턴 시내로 몰려 포위를 당했다.\n\n1780년 5월 12일에는 영국군의 헨리 클린턴 장군에게 항복하게 되었다. 이것은 독립 전쟁에서 대륙군 최대의 패배였다. 링컨은 항복 있어서, 전사의 명예를 부정당하게 되고 마음 속으로 화가 치밀었다. 링컨은 포로 교환 식으로 석방되었지만, 해제됐지만, 조사위원회에서는 어떤 허물도 문제되지 않았다. 링컨은 곧바로 워싱턴의 주력 부대로 복귀하여, 남부 버지니아로 가서 1781년 10월 19일의 요크타운에서 영국군의 항복까지 큰 역할을 했다. 이 항복은 콘월리스 경이 패배의 굴욕을 느끼고 워싱턴 장군에 직접 군도를 전달하는 항복 의식을 거부했다. 결국 부관인 찰스 오하라가 콘월리스 대신 서게 되었으며, 워싱턴의 부관이었던 링컨이 콘월리스의 군도를 받았다.


In [28]:
print(training_dataset['question'][IDX])
# print(retrieval.get_relevant_doc(training_dataset['question'][IDX]))
retrieval_list = retrieval.get_relevant_doc(training_dataset['question'][IDX], k=5)
for i in retrieval_list.tolist():
    print(training_dataset['context'][i])

링컨이 조지 워싱턴 장군에 합세하여 우익을 통솔했던 전투는?



tensor([[86.1941, 86.1700, 86.1245, 86.2547, 86.1179, 86.2094, 86.0226, 86.0526,
         86.0990, 86.0593, 86.2248, 86.0264, 85.9630, 86.3734, 86.0847, 86.0439,
         86.3819, 86.1201, 86.0742, 86.1609, 86.0990, 86.0352, 86.1289, 86.1771,
         86.1498, 86.1031, 86.0030, 86.1216, 86.0489, 86.2452, 86.1868, 86.3122,
         86.0979, 85.9766, 86.0921, 86.1825, 86.0275, 86.2494, 85.9991, 86.0983,
         86.2463, 86.1144, 85.9928, 86.1971, 86.0700, 86.0330, 86.1281, 86.2053,
         86.1489, 86.1334, 86.3268, 86.0768, 86.0330, 85.9015, 86.2053, 86.1359,
         85.9843, 86.1666, 86.0735, 86.3388, 86.1852, 86.2069, 86.2173, 86.0330,
         86.1078, 86.1198, 86.0115, 86.1644, 86.0768, 86.0023, 86.1617, 86.1920,
         85.9641, 86.0306, 86.1577, 86.0604, 86.3583, 85.9388, 85.9913, 86.2101,
         86.2272, 86.2305, 86.0813, 85.9851, 86.3177, 86.0586, 86.0588, 86.1189,
         86.3018, 86.1953, 86.1147, 86.3724, 86.0908, 85.8900, 86.1059, 86.0357,
         86.2109, 86.3115, 

In [ ]:
# 이거 에폭마다 eval 진행하게 함수 추가하기